# **Preparing datasets**

In [1]:
pip install chembl_webresource_client

     |████████████████████████████████| 61kB 3.7MB/s 
  Created wheel for chembl-webresource-client: filename=chembl_webresource_client-0.10.4-cp37-none-any.whl size=55640 sha256=38581fedae77d78b2e3b7e2e87920a41de3fcd70770b22ef9cb47ac28ffecad1
  Stored in directory: /root/.cache/pip/wheels/98/4c/16/bdb2ef9c937173e56b04d71fc1a6f98f0ff24992ae2e35daf3
Successfully built chembl-webresource-client


**Importing libraries**

In [2]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

**Search for Acetylcholinesterase**




In [3]:
# Target search for acetylcholinesterase
target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P22303', 'xref_name': None, 'xre...",Homo sapiens,Acetylcholinesterase,27.0,False,CHEMBL220,"[{'accession': 'P22303', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Homo sapiens,Cholinesterases; ACHE & BCHE,27.0,False,CHEMBL2095233,"[{'accession': 'P06276', 'component_descriptio...",SELECTIVITY GROUP,9606
2,[],Drosophila melanogaster,Acetylcholinesterase,17.0,False,CHEMBL2242744,"[{'accession': 'P07140', 'component_descriptio...",SINGLE PROTEIN,7227
3,"[{'xref_id': 'P04058', 'xref_name': None, 'xre...",Torpedo californica,Acetylcholinesterase,15.0,False,CHEMBL4780,"[{'accession': 'P04058', 'component_descriptio...",SINGLE PROTEIN,7787
4,"[{'xref_id': 'P21836', 'xref_name': None, 'xre...",Mus musculus,Acetylcholinesterase,15.0,False,CHEMBL3198,"[{'accession': 'P21836', 'component_descriptio...",SINGLE PROTEIN,10090
5,"[{'xref_id': 'P37136', 'xref_name': None, 'xre...",Rattus norvegicus,Acetylcholinesterase,15.0,False,CHEMBL3199,"[{'accession': 'P37136', 'component_descriptio...",SINGLE PROTEIN,10116
6,"[{'xref_id': 'O42275', 'xref_name': None, 'xre...",Electrophorus electricus,Acetylcholinesterase,15.0,False,CHEMBL4078,"[{'accession': 'O42275', 'component_descriptio...",SINGLE PROTEIN,8005
7,"[{'xref_id': 'P23795', 'xref_name': None, 'xre...",Bos taurus,Acetylcholinesterase,15.0,False,CHEMBL4768,"[{'accession': 'P23795', 'component_descriptio...",SINGLE PROTEIN,9913
8,[],Anopheles gambiae,Acetylcholinesterase,15.0,False,CHEMBL2046266,"[{'accession': 'Q869C3', 'component_descriptio...",SINGLE PROTEIN,7165
9,[],Bemisia tabaci,AChE2,15.0,False,CHEMBL2366409,"[{'accession': 'B3SST5', 'component_descriptio...",SINGLE PROTEIN,7038


**Select the first entry (single protein, rat) of the data for further investigation**

In [4]:
selected_entry = targets.target_chembl_id[5]
selected_entry

'CHEMBL3199'


Here, we will retrieve only bioactivity data for Human Acetylcholinesterase (CHEMBL220) that are reported as pChEMBL values.

In [5]:
activity = new_client.activity
filter_out = activity.filter(target_chembl_id=selected_entry).filter(standard_type="IC50")

In [6]:
df = pd.DataFrame.from_dict(filter_out)

In [7]:
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,32049,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC[N+](C)(CC)CC(=O)CCN1C(=O)CCC1=O.[I-],None,None,CHEMBL1123058,J. Med. Chem.,1985,"{'bei': '15.67', 'le': '0.30', 'lle': '3.42', ...",CHEMBL15886,None,CHEMBL1178213,4.00,False,http://www.openphacts.org/units/Nanomolar,17693,=,1,True,=,None,IC50,nM,None,100000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.1
1,None,32050,[],CHEMBL642842,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000221,tissue-based format,CC[N+](C)(CC)CC(=O)CCN1C(=O)CCC1=O.[I-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL15886,None,CHEMBL1178213,None,False,http://www.openphacts.org/units/Nanomolar,17693,=,1,True,=,None,IC50,nM,None,170000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.17
2,None,34486,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C[n+]1ccccc1CCC(O)CCl.[I-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL556129,None,CHEMBL1196197,None,False,http://www.openphacts.org/units/Nanomolar,17691,=,1,True,=,None,IC50,nM,None,200000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.2
3,None,34487,[],CHEMBL642842,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000221,tissue-based format,C[n+]1ccccc1CCC(O)CCl.[I-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL556129,None,CHEMBL1196197,None,False,http://www.openphacts.org/units/Nanomolar,17691,=,1,True,=,None,IC50,nM,None,200000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.2
4,None,38116,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C[N+](C)(C)CCCCl.[Br-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL15795,None,CHEMBL1178211,None,False,http://www.openphacts.org/units/Nanomolar,17688,=,1,True,=,None,IC50,nM,None,260000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758,None,19167595,[],CHEMBL4384296,Inhibition of rat cortex AChE using acetylthio...,B,None,None,BAO_0000190,BAO_0000019,assay format,COc1ccccc1CN(C)CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCC...,None,None,CHEMBL4382240,Bioorg Med Chem,2018,"{'bei': '7.50', 'le': '0.14', 'lle': '-0.99', ...",CHEMBL4475678,None,CHEMBL4475678,4.97,False,http://www.openphacts.org/units/Nanomolar,3198968,=,1,True,=,None,IC50,nM,None,10600.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,IC50,uM,UO_0000065,None,10.6
1759,None,19167596,[],CHEMBL4384296,Inhibition of rat cortex AChE using acetylthio...,B,None,None,BAO_0000190,BAO_0000019,assay format,CCN(CCCCCCNC(=O)c1cc(O)c

In [9]:
df.activity_id.count()

1763

Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [10]:
df.to_csv('rat_acetylcholinesterase_01_bioactivity_data.csv', index=False)

**Handling missing data**

Drop missing value for the **standard_value** and **canonical_smiles**

In [11]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,32049,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC[N+](C)(CC)CC(=O)CCN1C(=O)CCC1=O.[I-],None,None,CHEMBL1123058,J. Med. Chem.,1985,"{'bei': '15.67', 'le': '0.30', 'lle': '3.42', ...",CHEMBL15886,None,CHEMBL1178213,4.00,False,http://www.openphacts.org/units/Nanomolar,17693,=,1,True,=,None,IC50,nM,None,100000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.1
1,None,32050,[],CHEMBL642842,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000221,tissue-based format,CC[N+](C)(CC)CC(=O)CCN1C(=O)CCC1=O.[I-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL15886,None,CHEMBL1178213,None,False,http://www.openphacts.org/units/Nanomolar,17693,=,1,True,=,None,IC50,nM,None,170000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.17
2,None,34486,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C[n+]1ccccc1CCC(O)CCl.[I-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL556129,None,CHEMBL1196197,None,False,http://www.openphacts.org/units/Nanomolar,17691,=,1,True,=,None,IC50,nM,None,200000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.2
3,None,34487,[],CHEMBL642842,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000221,tissue-based format,C[n+]1ccccc1CCC(O)CCl.[I-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL556129,None,CHEMBL1196197,None,False,http://www.openphacts.org/units/Nanomolar,17691,=,1,True,=,None,IC50,nM,None,200000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.2
4,None,38116,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C[N+](C)(C)CCCCl.[Br-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL15795,None,CHEMBL1178211,None,False,http://www.openphacts.org/units/Nanomolar,17688,=,1,True,=,None,IC50,nM,None,260000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758,None,19167595,[],CHEMBL4384296,Inhibition of rat cortex AChE using acetylthio...,B,None,None,BAO_0000190,BAO_0000019,assay format,COc1ccccc1CN(C)CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCC...,None,None,CHEMBL4382240,Bioorg Med Chem,2018,"{'bei': '7.50', 'le': '0.14', 'lle': '-0.99', ...",CHEMBL4475678,None,CHEMBL4475678,4.97,False,http://www.openphacts.org/units/Nanomolar,3198968,=,1,True,=,None,IC50,nM,None,10600.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,IC50,uM,UO_0000065,None,10.6
1759,None,19167596,[],CHEMBL4384296,Inhibition of rat cortex AChE using acetylthio...,B,None,None,BAO_0000190,BAO_0000019,assay format,CCN(CCCCCCNC(=O)c1cc(O)c

In [12]:
len(df2.canonical_smiles.unique())

1448

Filter out the duplicate of **canonical_smiles**

In [13]:
df2_cs = df2.drop_duplicates(['canonical_smiles'])
df2_cs

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,32049,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC[N+](C)(CC)CC(=O)CCN1C(=O)CCC1=O.[I-],None,None,CHEMBL1123058,J. Med. Chem.,1985,"{'bei': '15.67', 'le': '0.30', 'lle': '3.42', ...",CHEMBL15886,None,CHEMBL1178213,4.00,False,http://www.openphacts.org/units/Nanomolar,17693,=,1,True,=,None,IC50,nM,None,100000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.1
2,None,34486,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C[n+]1ccccc1CCC(O)CCl.[I-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL556129,None,CHEMBL1196197,None,False,http://www.openphacts.org/units/Nanomolar,17691,=,1,True,=,None,IC50,nM,None,200000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.2
4,None,38116,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,C[N+](C)(C)CCCCl.[Br-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL15795,None,CHEMBL1178211,None,False,http://www.openphacts.org/units/Nanomolar,17688,=,1,True,=,None,IC50,nM,None,260000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.26
6,None,41591,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,BrCCCCCC[n+]1ccccc1.[Br-],None,None,CHEMBL1123058,J. Med. Chem.,1985,"{'bei': '17.36', 'le': '0.44', 'lle': '1.29', ...",CHEMBL276539,None,CHEMBL1183031,4.22,False,http://www.openphacts.org/units/Nanomolar,17700,=,1,True,=,None,IC50,nM,None,60000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.06
8,None,42915,[],CHEMBL642843,Effect on Acetylcholinesterase activity in rat...,B,None,None,BAO_0000190,BAO_0000219,cell-based format,CC[N+](C)(CC)CCc1ccncc1.[I-],Outside typical range,Values for this activity type are unusually la...,CHEMBL1123058,J. Med. Chem.,1985,None,CHEMBL278956,None,CHEMBL1183085,None,False,http://www.openphacts.org/units/Nanomolar,17696,=,1,True,=,None,IC50,nM,None,200000.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,I50,mM,UO_0000065,None,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756,None,19167593,[],CHEMBL4384296,Inhibition of rat cortex AChE using acetylthio...,B,None,None,BAO_0000190,BAO_0000019,assay format,CN(CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCCCCN(C)Cc2ccc...,None,None,CHEMBL4382240,Bioorg Med Chem,2018,"{'bei': '8.71', 'le': '0.16', 'lle': '-0.69', ...",CHEMBL4552250,None,CHEMBL4552250,5.25,False,http://www.openphacts.org/units/Nanomolar,3198966,=,1,True,=,None,IC50,nM,None,5600.0,CHEMBL3199,Rattus norvegicus,Acetylcholinesterase,10116,None,None,IC50,uM,UO_0000065,None,5.6
1757,None,19167594,[],CHEMBL4384296,Inhibition of rat cortex AChE using acetylthio...,B,None,None,BAO_0000190,BAO_0000019,assay format,CCN(CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCCCCN(CC)Cc2c...,None,No

# **Data pre-processing of the bioactivity data**

Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame

In [14]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_cs[selection]
df3

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL15886,CC[N+](C)(CC)CC(=O)CCN1C(=O)CCC1=O.[I-],100000.0
2,CHEMBL556129,C[n+]1ccccc1CCC(O)CCl.[I-],200000.0
4,CHEMBL15795,C[N+](C)(C)CCCCl.[Br-],260000.0
6,CHEMBL276539,BrCCCCCC[n+]1ccccc1.[Br-],60000.0
8,CHEMBL278956,CC[N+](C)(CC)CCc1ccncc1.[I-],200000.0
...,...,...,...
1756,CHEMBL4552250,CN(CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCCCCN(C)Cc2ccc...,5600.0
1757,CHEMBL4520323,CCN(CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCCCCN(CC)Cc2c...,4600.0
1758,CHEMBL4475678,COc1ccccc1CN(C)CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCC...,10600.0
1759,CHEMBL4451935,CCN(CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCCCCN(CC)Cc2c...,1680.0


Sava dataframe to CSV file.

In [15]:
df3.to_csv('rat_acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)

**Labeling compounds as being active, inactive or intermediate**

The bioactivity data is in the IC50 unit. Compounds having values of **less than 1000 nM** will be considered to be **active** while those **greater than 10,000 nM** will be considered to be **inactive**. As for those values in **between 1,000 and 10,000** nM will be referred to as **intermediate**.

In [16]:
df4 = pd.read_csv('rat_acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

In [17]:
bioactivity = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity.append("inactive")
  elif float(i) <= 1000:
    bioactivity.append("active")
  else:
    bioactivity.append("intermediate")

In [18]:
bioactivity_class = pd.Series(bioactivity, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL15886,CC[N+](C)(CC)CC(=O)CCN1C(=O)CCC1=O.[I-],100000.00,inactive
1,CHEMBL556129,C[n+]1ccccc1CCC(O)CCl.[I-],200000.00,inactive
2,CHEMBL15795,C[N+](C)(C)CCCCl.[Br-],260000.00,inactive
3,CHEMBL276539,BrCCCCCC[n+]1ccccc1.[Br-],60000.00,inactive
4,CHEMBL278956,CC[N+](C)(CC)CCc1ccncc1.[I-],200000.00,inactive
...,...,...,...,...
1443,CHEMBL4552250,CN(CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCCCCN(C)Cc2ccc...,5600.00,intermediate
1444,CHEMBL4520323,CCN(CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCCCCN(CC)Cc2c...,4600.00,intermediate
1445,CHEMBL4475678,COc1ccccc1CN(C)CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCC...,10600.00,inactive
1446,CHEMBL4451935,CCN(CCCCCCNC(=O)c1cc(O)c(C(=O)NCCCCCCN(CC)Cc2c...,1680.00,intermediate


In [19]:
df5.to_csv('rat_acetylcholinesterase_03_bioactivity_data_curated.csv', index=False)

In [ ]:
! zip acetylcholinesterase.zip *.csv

  adding: acetylcholinesterase_01_bioactivity_data.csv (deflated 91%)
  adding: acetylcholinesterase_02_bioactivity_data_preprocessed.csv (deflated 81%)
  adding: acetylcholinesterase_03_bioactivity_data_curated.csv (deflated 82%)
